# Oefeningen Weblog

We willen analyses kunnen uitvoeren op de logs van een webserver ("UofS_access_log.small"). Om dit te kunnen doen, moet je de relevante gegevens in een dataframe krijgen. Dit moet een automatisch proces zijn zodat ook andere logfiles ingeladen kan worden.

Volgende zaken dienen te gebeuren. Het originele dataframe moet herwerkt worden zodat enkel deze kolommen overblijven:

- domain: bevat de adressen van de clients die een request stuurden
- timestamp: is een datetime veld (POSIXct) dat het tijdstip van de request weergeeft
- resource: toont de resource die werd opgevraagd
- repsponse_code: geeft de HTTP response code die werd terug gegeven door de server
- response_length: geeft de lengte aan van de HTTP response

We gaan dit stap voor stap opbouwen.

Importeer hier alle nodige bibliotheken:

In [5]:
import pandas as pd

### Inlezen van de gegevens
Lees het dataframe in.  Kijk zelf na wat de seperator is.  Foutieve rijen mag je skippen.  Er is geen header! Het bestand gebruikt de "latin" encodering voor karakters.

In [6]:
df = pd.read_csv("data/UofS_access_log.small", sep=" ", header=None, on_bad_lines='skip', encoding="latin")
display(df)

,0,1,2,3,4,5,6,7
0,cad49.cadvision.com,-,-,[01/Jun/1995:00:53:19,-0600],GET /~lowey/webville/icons/blank_32.gif,200,167
1,130.89.250.24,-,-,[01/Jun/1995:02:45:12,-0600],GET /~lowey/webville/icons/south_32.gif,200,210
2,130.54.25.198,-,-,[01/Jun/1995:03:29:56,-0600],GET /~macphed/finite/fe_resources/node92.html,200,1668
3,148.81.17.41,-,-,[01/Jun/1995:04:02:17,-0600],GET /~friesend/tolkien/rootpage.html,200,461
4,anumsun6.univ-st-etienne.fr,-,-,[01/Jun/1995:04:40:30,-0600],GET /~macphed/finite/fe_resources/node58.html,200,1707
...,...,...,...,...,...,...,...,...
48166,duke.usask.ca,-,-,[31/Dec/1995:23:07:50,-0600],GET /images/comments_50.gif HTTP/1.0,200,388
48167,sask.usask.ca,-,-,[31/Dec/1995:23:16:16,-0600],GET / HTTP/1.0,200,3856
48168,144.92.13.6,-,-,[31/Dec/1995:23:25:53,-0600],GET /~ladd/virginia_pisu.html,200,915
48169,ccc055.canuck.com,-,-,[31/Dec/1995:23:32:35,-0600],GET /mlookup/white.html HTTP/1.0,200,728


Hoeveel lijnen staan er in dit data frame?

In [7]:
display(len(df))

48171

Kopieer alle waarden uit de eerste kolom in een variabele "domain".
Kopieer alle waarden uit de zevende kolom in een variabele "response_code"
Kopieer alle waarden uit de achtste kolom in een variabele "response_length"

In [8]:
domain = df.iloc[:,0]
response_code = df.iloc[:,6]
response_length = df.iloc[:,7]

Kijk na of de datatypes van "domain", "response_code" en "response_length" juist zijn.  We verwachten respectievelijk: strings, getallen en getallen.  Converteer de variabelen indien nodig.  Als er ongeldige waarden zijn, zouden deze omgezet moeten worden naar NaN.

In [18]:
response_code = pd.to_numeric(response_code, errors="coerce")
response_length = pd.to_numeric(response_length, errors="coerce")

,0,1,2,3,4,5,6,7
0,cad49.cadvision.com,-,-,[01/Jun/1995:00:53:19,-0600],GET /~lowey/webville/icons/blank_32.gif,200,167
1,130.89.250.24,-,-,[01/Jun/1995:02:45:12,-0600],GET /~lowey/webville/icons/south_32.gif,200,210
2,130.54.25.198,-,-,[01/Jun/1995:03:29:56,-0600],GET /~macphed/finite/fe_resources/node92.html,200,1668
3,148.81.17.41,-,-,[01/Jun/1995:04:02:17,-0600],GET /~friesend/tolkien/rootpage.html,200,461
4,anumsun6.univ-st-etienne.fr,-,-,[01/Jun/1995:04:40:30,-0600],GET /~macphed/finite/fe_resources/node58.html,200,1707
...,...,...,...,...,...,...,...,...
48166,duke.usask.ca,-,-,[31/Dec/1995:23:07:50,-0600],GET /images/comments_50.gif HTTP/1.0,200,388
48167,sask.usask.ca,-,-,[31/Dec/1995:23:16:16,-0600],GET / HTTP/1.0,200,3856
48168,144.92.13.6,-,-,[31/Dec/1995:23:25:53,-0600],GET /~ladd/virginia_pisu.html,200,915
48169,ccc055.canuck.com,-,-,[31/Dec/1995:23:32:35,-0600],GET /mlookup/white.html HTTP/1.0,200,728


Hoeveel NaN waarden zitten er in response_length?

In [10]:
display(response_length.isna().sum())

897

Hoeveel procent is dat van alle rijen?

In [11]:
print(round(response_length.isna().sum() / len(response_length * 100),2), "%")

0.02 %


De timestamp staat verspreid in kolommen 3 (datum en uur) en 4 (timezone). Voeg deze samen tot één string. Plaats het resultaat in een variabele "timestamp".

In [12]:
timestamp = df.iloc[:,3] + df.iloc[:,4]
display(timestamp)

0        [01/Jun/1995:00:53:19-0600]
1        [01/Jun/1995:02:45:12-0600]
2        [01/Jun/1995:03:29:56-0600]
3        [01/Jun/1995:04:02:17-0600]
4        [01/Jun/1995:04:40:30-0600]
                    ...             
48166    [31/Dec/1995:23:07:50-0600]
48167    [31/Dec/1995:23:16:16-0600]
48168    [31/Dec/1995:23:25:53-0600]
48169    [31/Dec/1995:23:32:35-0600]
48170    [31/Dec/1995:23:44:25-0600]
Length: 48171, dtype: object


Maak een variabele "resource" die alle resources bevat (in de 6e kolom). Verwijder de 'GET' en 'HTTP/1.0' die soms in het begin en op het einde staan.

In [19]:
resource = df.iloc[:,5]

resource = resource.str.replace("GET ", "", regex=False)
resource = resource.str.replace(" HTTP/1.0", "", regex=False)

# TEST
print(resource)

0              /~lowey/webville/icons/blank_32.gif
1              /~lowey/webville/icons/south_32.gif
2        /~macphed/finite/fe_resources/node92.html
3                 /~friesend/tolkien/rootpage.html
4        /~macphed/finite/fe_resources/node58.html
                           ...                    
48166                      /images/comments_50.gif
48167                                            /
48168                    /~ladd/virginia_pisu.html
48169                          /mlookup/white.html
48170                         /~scottp/calgary.txt
Name: 5, Length: 48171, dtype: object

Maak nu een dataframe met naam "log" waarin de kolommen "domain", "timestamp", "resource", "response_code" en "response_length" staan.  De waarden kan je uit de gemaakte variabelen halen.

In [14]:
log = pd.DataFrame({"domain":domain, "timestamp":timestamp, "resource":resource, "response_code":response_code,"response_length":response_length})

# TEST
display(log)

,domain,timestamp,resource,response_code,response_length
0,cad49.cadvision.com,[01/Jun/1995:00:53:19-0600],/~lowey/webville/icons/blank_32.gif,200,167.0
1,130.89.250.24,[01/Jun/1995:02:45:12-0600],/~lowey/webville/icons/south_32.gif,200,210.0
2,130.54.25.198,[01/Jun/1995:03:29:56-0600],/~macphed/finite/fe_resources/node92.html,200,1668.0
3,148.81.17.41,[01/Jun/1995:04:02:17-0600],/~friesend/tolkien/rootpage.html,200,461.0
4,anumsun6.univ-st-etienne.fr,[01/Jun/1995:04:40:30-0600],/~macphed/finite/fe_resources/node58.html,200,1707.0
...,...,...,...,...,...
48166,duke.usask.ca,[31/Dec/1995:23:07:50-0600],/images/comments_50.gif,200,388.0
48167,sask.usask.ca,[31/Dec/1995:23:16:16-0600],/,200,3856.0
48168,144.92.13.6,[31/Dec/1995:23:25:53-0600],/~ladd/virginia_pisu.html,200,915.0
48169,ccc055.canuck.com,[31/Dec/1995:23:32:35-0600],/mlookup/white.html,200,728.0


Verwijder nu alle rijen uit je dataframe waar een ontbrekende waarde in voorkomt.

In [15]:
log = log.dropna()

# TEST
display(log)

,domain,timestamp,resource,response_code,response_length
0,cad49.cadvision.com,[01/Jun/1995:00:53:19-0600],/~lowey/webville/icons/blank_32.gif,200,167.0
1,130.89.250.24,[01/Jun/1995:02:45:12-0600],/~lowey/webville/icons/south_32.gif,200,210.0
2,130.54.25.198,[01/Jun/1995:03:29:56-0600],/~macphed/finite/fe_resources/node92.html,200,1668.0
3,148.81.17.41,[01/Jun/1995:04:02:17-0600],/~friesend/tolkien/rootpage.html,200,461.0
4,anumsun6.univ-st-etienne.fr,[01/Jun/1995:04:40:30-0600],/~macphed/finite/fe_resources/node58.html,200,1707.0
...,...,...,...,...,...
48166,duke.usask.ca,[31/Dec/1995:23:07:50-0600],/images/comments_50.gif,200,388.0
48167,sask.usask.ca,[31/Dec/1995:23:16:16-0600],/,200,3856.0
48168,144.92.13.6,[31/Dec/1995:23:25:53-0600],/~ladd/virginia_pisu.html,200,915.0
48169,ccc055.canuck.com,[31/Dec/1995:23:32:35-0600],/mlookup/white.html,200,728.0


Zoek de rij(en) met de grootste response_length:

In [16]:
print(log.response_length.max)

<bound method NDFrame._add_numeric_operations.<locals>.max of 0          167.0
1          210.0
2         1668.0
3          461.0
4         1707.0
          ...   
48166      388.0
48167     3856.0
48168      915.0
48169      728.0
48170    24744.0
Name: response_length, Length: 47274, dtype: float64>


Bewaar het resultaat in een CSV-bestand "log_resultaat.csv".  Gebruik ',' als separator en "." voor decimale getallen.

In [17]:
log.to_csv("log_resultaat.csv", sep=",", decimal=".")

Probeer het bestand in een spreadsheet te importeren.